In [1]:
import numpy as np
from scipy.linalg import lapack
from scipy.special import expit
from scipy.stats import chi2
import pandas as pd
from scipy.special import expit
from firthlogist import FirthLogisticRegression
from numba import njit

In [106]:
# Q = np.random.rand(20000, 17)
# def scipy_qr(Q):
#     qr, tau, _, _ = lapack.dgeqrf(Q)
#     x, _, _ = lapack.dorgqr(qr, tau)
#     return x

# %timeit np.linalg.qr(Q)
# %timeit scipy_qr(Q)

preds = np.random.rand(20000)
X = np.random.rand(20000, 17)
def old(preds, X):
    rootW = np.sqrt(preds * (1 - preds))
    XW = rootW[:, np.newaxis] * X
    qr, tau, _, _ = lapack.dgeqrf(XW)
    Q, _, _ = lapack.dorgqr(qr, tau)
    hat = np.einsum("ij,ij->i", Q, Q)
    return hat

def new(preds, X):
    rootW = np.sqrt(preds * (1 - preds))
#     XW = rootW[:, np.newaxis] * X
    XtW = np.multiply(X.T, rootW)
    WX = np.multiply(rootW[:, np.newaxis], X)
    XtWX = XtW @ X
#     print((XtWX @ XtW).shape)
    hat_mtx = XW @ np.linalg.solve(XtWX, XtW)
    print(XW.shape)
    print(hat_mtx.shape)
#     fim = XW @ XW.T
#     print(np.allclose(XtWX, fim))
#     x = np.linalg.solve(XtWX, X.T)
#     Q_ = np.einsum('ij,ji->i', X,hat_mtx) 
#     return Q_
new(preds, X)
# np.allclose(old(preds, X), new(preds, X))

(20000, 17)
(20000, 20000)


In [99]:
m = 200
n = 500000

np.random.seed(123)
X = np.random.random((n,m))
W_diag = np.random.random(n)            # C -> dense vector


lhs = np.multiply(X.T, W_diag).dot(X)   # C (+A)
x = np.linalg.solve(lhs, X.T)           # B
print(lhs.shape)
print(X.T.shape)
print(x.shape)

(200, 200)
(200, 500000)
(200, 500000)


In [68]:
# np.multiply(X.T, rootW)
# rootW[:, np.newaxis] * X
np.multiply(X, rootW[:, np.newaxis])

array([[0.04783116, 0.17454019, 0.10998432, ..., 0.03930919, 0.18292077,
        0.16507641],
       [0.08172705, 0.11543252, 0.09174067, ..., 0.09862448, 0.10468953,
        0.01189124],
       [0.41615762, 0.27235266, 0.34298398, ..., 0.13732365, 0.16597937,
        0.1797666 ],
       ...,
       [0.30277286, 0.27870828, 0.3397394 , ..., 0.296657  , 0.19384353,
        0.14704407],
       [0.16671844, 0.13123987, 0.19758824, ..., 0.38240628, 0.19017203,
        0.17584479],
       [0.27519537, 0.1272537 , 0.21710494, ..., 0.20821412, 0.02053744,
        0.213298  ]])

In [55]:
print(old(preds, X))
print(new(preds, X))

[0.00047494 0.0014886  0.00075984 ... 0.00107517 0.00128043 0.00021424]
[0.00316248 0.00255296 0.00134731 ... 0.00211485 0.00230587 0.00208776]


In [43]:
from transonic import jit, wait_for_all_extensions
import numpy as np

Q = np.random.rand(20000, 17)

def transonic_sum(Q):
    hat = np.sum(Q*Q, axis=1)
    return hat

def manual(Q):
    res = np.zeros(Q.shape[0])
    for idx, row in enumerate(Q):
        rowsum = 0
        for col in row:
            rowsum += col*col
        res[idx] = rowsum
    return res
    
transonic_sum_xsimd = jit(native=True, xsimd=True)(transonic_sum)
transonic_sum_noxsimd = jit(native=True, xsimd=False)(transonic_sum)
transonic_manual_xsimd = jit(native=True, xsimd=True)(manual)
transonic_manual_noxsimd = jit(native=True, xsimd=False)(manual)
transonic_sum_xsimd(Q)
transonic_sum_noxsimd(Q)
transonic_manual_xsimd(Q)
transonic_manual_noxsimd(Q)

Schedule pythranization: transonic_sum.py
Schedule pythranization: transonic_sum.py
Schedule pythranization: manual.py


Pythranizing /home/jon/.transonic/pythran/__jit__/__ipython__227c27df279439a4a4c3b3cf22aedc2f/transonic_sum.py
lock file /home/jon/.transonic/pythran/__jit__/__ipython__227c27df279439a4a4c3b3cf22aedc2f/transonic_sum_1516e6def7e005fb004ba29d955a06be_cee621790ba67ed1e2c2fa0a6ee38726.lock present: waiting for completion of the compilation


Schedule pythranization: manual.py


Pythranizing /home/jon/.transonic/pythran/__jit__/__ipython__227c27df279439a4a4c3b3cf22aedc2f/manual.py
lock file /home/jon/.transonic/pythran/__jit__/__ipython__227c27df279439a4a4c3b3cf22aedc2f/manual_ca769040ca18646de8ac8e0599cbb244_d87f1dabdcce1f5ca567446f575ebe1b.lock present: waiting for completion of the compilation


array([4.46701855, 4.25030818, 5.51264391, ..., 7.23127325, 5.65918928,
       7.26287235])

File /home/jon/.transonic/pythran/__jit__/__ipython__227c27df279439a4a4c3b3cf22aedc2f/manual_ca769040ca18646de8ac8e0599cbb244_d87f1dabdcce1f5ca567446f575ebe1b.cpython-310-x86_64-linux-gnu.so created by pythran
File /home/jon/.transonic/pythran/__jit__/__ipython__227c27df279439a4a4c3b3cf22aedc2f/transonic_sum_1516e6def7e005fb004ba29d955a06be_cee621790ba67ed1e2c2fa0a6ee38726.cpython-310-x86_64-linux-gnu.so created by pythran


In [45]:
%timeit np.einsum("ij,ij->i", Q, Q)
%timeit np.sum(np.square(Q), axis=1)
# %timeit np.sum(Q*Q, axis=1)
# %timeit transonic_sum_xsimd(Q)
# %timeit transonic_sum_noxsimd(Q)
# # %timeit manual(Q)
# %timeit transonic_manual_xsimd(Q)
# %timeit transonic_manual_noxsimd(Q)

92.7 µs ± 55 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
404 µs ± 1.22 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [4]:
from transonic import jit, wait_for_all_extensions
import numpy as np

def loglike(y, preds):
    return np.sum(y * np.log(preds) + (1 - y) * np.log(1 - preds))

loglike_pythran = jit(native=True, xsimd=False)(loglike)
loglike_pythran_xsimd = jit(native=True, xsimd=True)(loglike)
loglike_pythran(np.array([0.,1.,1.,0.]), np.array([0.,1.,1.,0.]))
loglike_pythran_xsimd(np.array([0.,1.,1.,0.]), np.array([0.,1.,1.,0.]))

Schedule pythranization: loglike.py
/tmp/ipykernel_31521/2569637500.py:5: RuntimeWarning: divide by zero encountered in log
  return np.sum(y * np.log(preds) + (1 - y) * np.log(1 - preds))
/tmp/ipykernel_31521/2569637500.py:5: RuntimeWarning: invalid value encountered in multiply
  return np.sum(y * np.log(preds) + (1 - y) * np.log(1 - preds))
Schedule pythranization: loglike.py


nan

Pythranizing /home/jon/.transonic/pythran/__jit__/__ipython__ca578a667d4d130c459901814cffe800/loglike.py
lock file /home/jon/.transonic/pythran/__jit__/__ipython__ca578a667d4d130c459901814cffe800/loglike_1a2a3d52460439e65005b1b042015697_b3d43e242266d2bba4f32ea82224270e.lock present: waiting for completion of the compilation
File /home/jon/.transonic/pythran/__jit__/__ipython__ca578a667d4d130c459901814cffe800/loglike_1a2a3d52460439e65005b1b042015697_b3d43e242266d2bba4f32ea82224270e.cpython-310-x86_64-linux-gnu.so created by pythran


In [8]:
@njit
def numba_jit(y, preds):
    return np.sum(y * np.log(preds) + (1 - y) * np.log(1 - preds))

a = np.random.rand(1000000)
b = np.random.rand(1000000)
numba_jit(a, b)
%timeit loglike(a, b)
%timeit numba_jit(a, b)
%timeit loglike_pythran(a, b)
%timeit loglike_pythran_xsimd(a, b)

10.7 ms ± 114 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
7.82 ms ± 11.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
7.82 ms ± 1.66 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
7.83 ms ± 2.91 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [77]:
a = np.random.rand(1000)
b = np.random.rand(1000)
%timeit loglike_jit(a, b)
%timeit loglike(a, b)

7.27 µs ± 118 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
13.3 µs ± 41.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [85]:
import warnings
from copy import deepcopy
from importlib.resources import open_text
from math import sqrt

import numpy as np
from scipy.linalg import lapack
from scipy.special import expit
from scipy.stats import chi2
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.exceptions import ConvergenceWarning
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.multiclass import check_classification_targets
from sklearn.utils.validation import check_is_fitted
from tabulate import tabulate
# from loglike import loglike


class FirthLogisticRegression(BaseEstimator, ClassifierMixin):
    """
    Logistic regression with Firth's bias reduction method.

    This is based on the implementation in the `logistf` R package. Please see the
    `logistf` reference and Heinze & Schemper (2002) for details about the procedure.

    Parameters
    ----------
    max_iter
        The maximum number of Newton-Raphson iterations.
    max_halfstep
        The maximum number of step-halvings in one Newton-Raphson iteration.
    max_stepsize
        The maximum step size - for each coefficient, the step size is forced to
        be less than max_stepsize.
    pl_max_iter
        The maximum number of Newton-Raphson iterations for finding profile likelihood
        confidence intervals.
    pl_max_halfstep
        The maximum number of step-halvings in one iteration for finding profile
        likelihood confidence intervals.
    pl_max_stepsize
        The maximum step size while finding PL confidence intervals.
    tol
        Convergence tolerance for stopping.
    fit_intercept
        Specifies if intercept should be added.
    skip_lrt
        If True, p-values will not be calculated. Calculating the p-values can be
        time-consuming since the fitting procedure is repeated for each coefficient.
    skip_ci
        If True, confidence intervals will not be calculated. Calculating the confidence
        intervals via profile likelihoood is time-consuming.
    alpha
        Significance level (confidence interval = 1-alpha). 0.05 as default for 95% CI.

    Attributes
    ----------
    bse_
        Standard errors of the coefficients.
    classes_
        A list of the class labels.
    ci_
        The fitted profile likelihood confidence intervals.
    coef_
        The coefficients of the features.
    intercept_
        Fitted intercept. If `fit_intercept = False`, the intercept is set to zero.
    loglik_
        Fitted penalized log-likelihood.
    n_iter_
        Number of Newton-Raphson iterations performed.
    pvals_
        p-values calculated by penalized likelihood ratio tests.

    References
    ----------
    Firth D (1993). Bias reduction of maximum likelihood estimates.
    Biometrika 80, 27–38.

    Heinze G, Schemper M (2002). A solution to the problem of separation in logistic
    regression. Statistics in Medicine 21: 2409-2419.
    """

    def __init__(
        self,
        max_iter=25,
        max_halfstep=0,
        max_stepsize=5,
        pl_max_iter=100,
        pl_max_halfstep=0,
        pl_max_stepsize=5,
        tol=0.0001,
        fit_intercept=True,
        skip_lrt=False,
        skip_ci=False,
        alpha=0.05,
        jit = False
    ):
        self.max_iter = max_iter
        self.max_stepsize = max_stepsize
        self.max_halfstep = max_halfstep
        self.pl_max_iter = pl_max_iter
        self.pl_max_halfstep = pl_max_halfstep
        self.pl_max_stepsize = pl_max_stepsize
        self.tol = tol
        self.fit_intercept = fit_intercept
        self.skip_lrt = skip_lrt
        self.skip_ci = skip_ci
        self.alpha = alpha
        self.jit = jit

    def _more_tags(self):
        return {"binary_only": True}

    def _validate_input(self, X, y):
        if self.max_iter < 0:
            raise ValueError(
                f"Maximum number of iterations must be positive; "
                f"got max_iter={self.max_iter}"
            )
        if self.max_halfstep < 0:
            raise ValueError(
                f"Maximum number of step-halvings must >= 0; "
                f"got max_halfstep={self.max_iter}"
            )
        if self.tol < 0:
            raise ValueError(
                f"Tolerance for stopping criteria must be positive; got tol={self.tol}"
            )
        X, y = self._validate_data(X, y, dtype=np.float64, ensure_min_samples=2)
        check_classification_targets(y)

        self.classes_ = np.unique(y)
        if len(self.classes_) != 2:
            raise ValueError(f"Got {len(self.classes_)} - only 2 classes supported.")
        y = LabelEncoder().fit_transform(y).astype(X.dtype, copy=False)

        return X, y

    def fit(self, X, y):
        X, y = self._validate_input(X, y)
        if self.fit_intercept:
            X = np.hstack((X, np.ones((X.shape[0], 1))))

        self.coef_, self.loglik_, self.n_iter_ = _firth_newton_raphson(
            X, y, self.max_iter, self.max_stepsize, self.max_halfstep, self.tol, jit=self.jit
        )

        self.bse_ = _bse(X, self.coef_)

        if not self.skip_ci:
            self.ci_ = np.column_stack(
                [
                    _profile_likelihood_ci(
                        X=X,
                        y=y,
                        side=side,
                        fitted_coef=self.coef_,
                        full_loglik=self.loglik_,
                        max_iter=self.pl_max_iter,
                        max_stepsize=self.pl_max_stepsize,
                        max_halfstep=self.pl_max_halfstep,
                        tol=self.tol,
                        alpha=0.05,
                        jit=self.jit
                    )
                    for side in [-1, 1]
                ]
            )

        # penalized likelihood ratio tests
        if not self.skip_lrt:
            pvals = []
            # mask is 1-indexed because of `if mask` check in _get_XW()
            for mask in range(1, self.coef_.shape[0] + 1):
                _, null_loglik, _ = _firth_newton_raphson(
                    X,
                    y,
                    self.max_iter,
                    self.max_stepsize,
                    self.max_halfstep,
                    self.tol,
                    mask,
                    jit=self.jit
                )
                pvals.append(_lrt(self.loglik_, null_loglik))
            self.pvals_ = np.array(pvals)

        if self.fit_intercept:
            self.intercept_ = self.coef_[-1]
            self.coef_ = self.coef_[:-1]
        else:
            self.intercept_ = 0

        return self

    def summary(self, xname=None, tablefmt="simple"):
        """
        Prints a summary table.

        Parameters
        ----------
        xname
            Names for the X variables. Default is x1, x2, ... Must match the number of
            parameters in the model.
        tablefmt
            `tabulate` table format for output. Please see the documentation for
            `tabulate` for options.
        """
        check_is_fitted(self)
        if xname and len(xname) != len(self.coef_):
            raise ValueError(
                f"Length of xname ({len(xname)}) does not match the number of "
                f"parameters in the model ({len(self.coef_)})"
            )

        if not xname:
            xname = [f"x{i}" for i in range(1, len(self.coef_) + 1)]

        coef = list(self.coef_)
        if self.fit_intercept:
            xname.append("Intercept")
            coef.append(self.intercept_)

        headers = [
            "",
            "coef",
            "std err",
            f"[{self.alpha/2}",
            f"{1-self.alpha/2}]",
            "p-value",
        ]
        table = zip(xname, coef, self.bse_, self.ci_[:, 0], self.ci_[:, 1], self.pvals_)
        table = tabulate(table, headers, tablefmt=tablefmt)
        table += "\n\n"
        table += f"Log-Likelihood: {round(self.loglik_, 4)}\n"
        table += f"Newton-Raphson iterations: {self.n_iter_}\n"
        print(table)
        return table

    def decision_function(self, X):
        check_is_fitted(self)
        X = self._validate_data(X, reset=False)
        scores = X @ self.coef_ + self.intercept_
        return scores

    def predict(self, X):
        decision = self.decision_function(X)
        if len(decision.shape) == 1:
            indices = (decision > 0).astype(int)
        else:
            indices = decision.argmax(axis=1)
        return self.classes_[indices]

    def predict_proba(self, X):
        decision = self.decision_function(X)
        if decision.ndim == 1:
            decision = np.c_[-decision, decision]
        proba = expit(decision)
        return proba


def _firth_newton_raphson(X, y, max_iter, max_stepsize, max_halfstep, tol, mask=None, jit=False):
    # see logistf reference manual for explanation of procedure
    coef = np.zeros(X.shape[1])
    for iter in range(1, max_iter + 1):
        preds = expit(X @ coef)
        XW = _get_XW(X, preds, mask)

        fisher_info_mtx = XW.T @ XW
        hat = _hat_diag(XW)
        U_star = np.matmul(X.T, y - preds + np.multiply(hat, 0.5 - preds))
        step_size = np.linalg.lstsq(fisher_info_mtx, U_star, rcond=None)[0]
        # if mask:
        #     step_size[mask] = 0

        # step-halving
        mx = np.max(np.abs(step_size)) / max_stepsize
        if mx > 1:
            step_size = step_size / mx  # restrict to max_stepsize
        coef_new = coef + step_size
        preds_new = expit(X @ coef_new)
        loglike = _loglikelihood(X, y, preds, jit)
        loglike_new = _loglikelihood(X, y, preds_new, jit)
        steps = 0
        while loglike < loglike_new:
            step_size *= 0.5
            coef_new = coef + step_size
            preds_new = expit(X @ coef_new)
            loglike_new = _loglikelihood(X, y, preds_new, jit)
            steps += 1
            if steps == max_halfstep:
                warning_msg = "Step-halving failed to converge."
                warnings.warn(warning_msg, ConvergenceWarning, stacklevel=2)
                return coef_new, -loglike_new, iter

        if iter > 1 and np.linalg.norm(coef_new - coef) < tol:
            return coef_new, -loglike_new, iter

        coef += step_size
    warning_msg = (
        "Firth logistic regression failed to converge. Try increasing max_iter."
    )
    warnings.warn(warning_msg, ConvergenceWarning, stacklevel=2)
    return coef, -loglike_new, max_iter


def _loglikelihood(X, y, preds, jit):
    # penalized log-likelihood
    XW = _get_XW(X, preds)
    fisher_info_mtx = XW.T @ XW
    penalty = 0.5 * np.log(np.linalg.det(fisher_info_mtx))
    if not jit:
        return -1 * (np.sum(y * np.log(preds) + (1 - y) * np.log(1 - preds)) + penalty)
    return -1 * (loglike_jit(y, preds) + penalty)

def _get_XW(X, preds, mask=None):
    # mask is 1-indexed because 0 == None
    rootW = np.sqrt(preds * (1 - preds))
    XW = rootW[:, np.newaxis] * X

    # is this equivalent??
    # https://github.com/georgheinze/logistf/blob/master/src/logistf.c#L150-L159
    if mask:
        XW[:, mask - 1] = 0
    return XW


def _get_aug_XW(X, preds, hats):
    rootW = np.sqrt(preds * (1 - preds) * (1 + hats))
    XW = rootW[:, np.newaxis] * X
    return XW


def _hat_diag(XW):
    # Get diagonal elements of the hat matrix
    # Q = np.linalg.qr(XW, mode="reduced")[0]
    qr, tau, _, _ = lapack.dgeqrf(XW)
    Q, _, _ = lapack.dorgqr(qr, tau)
    hat = np.einsum("ij,ij->i", Q, Q)
    return hat


def _bse(X, coefs):
    # se in logistf is diag(object$var) ^ 0.5, where var is the covariance matrix,
    # which is the inverse of the observed fisher information matrix
    # https://stats.stackexchange.com/q/68080/343314
    preds = expit(X @ coefs)
    XW = _get_XW(X, preds)
    fisher_info_mtx = XW.T @ XW
    return np.sqrt(np.diag(np.linalg.pinv(fisher_info_mtx)))


def _lrt(full_loglik, null_loglik):
    # in logistf: 1-pchisq(2*(fit.full$loglik-fit.i$loglik),1)
    lr_stat = 2 * (full_loglik - null_loglik)
    p_value = chi2.sf(lr_stat, df=1)
    return p_value


def _predict(X, coef):
    preds = expit(X @ coef)
    np.clip(preds, a_min=1e-15, a_max=1 - 1e-15, out=preds)
    return preds


def _profile_likelihood_ci(
    X,
    y,
    side,
    fitted_coef,
    full_loglik,
    max_iter,
    max_stepsize,
    max_halfstep,
    tol,
    alpha,
    jit
):
    LL0 = full_loglik - chi2.ppf(1 - alpha, 1) / 2
    ci = []
    for coef_idx in range(fitted_coef.shape[0]):
        coef = deepcopy(fitted_coef)
        for iter in range(1, max_iter + 1):
            # preds = expit(X @ coef)
            preds = _predict(X, coef)
            loglike = -_loglikelihood(X, y, preds, jit)
            XW = _get_XW(X, preds)
            hat = _hat_diag(XW)
            XW = _get_aug_XW(X, preds, hat)  # augmented data using hat diag
            fisher_info_mtx = XW.T @ XW
            U_star = np.matmul(X.T, y - preds + np.multiply(hat, 0.5 - preds))
            # https://github.com/georgheinze/logistf/blob/master/src/logistf.c#L780-L781
            inv_fisher = np.linalg.pinv(fisher_info_mtx)
            tmp1x1 = U_star @ np.negative(inv_fisher) @ U_star
            underRoot = (
                -2 * ((LL0 - loglike) + 0.5 * tmp1x1) / (inv_fisher[coef_idx, coef_idx])
            )
            lambda_ = 0 if underRoot < 0 else side * sqrt(underRoot)
            U_star[coef_idx] += lambda_

            step_size = np.linalg.lstsq(fisher_info_mtx, U_star, rcond=None)[0]
            mx = np.max(np.abs(step_size)) / max_stepsize
            if mx > 1:
                step_size = step_size / mx  # restrict to max_stepsize
            coef += step_size
            loglike_old = deepcopy(loglike)

            for halfs in range(1, max_halfstep + 1):
                # preds = expit(X @ coef)
                preds = _predict(X, coef)
                loglike = -_loglikelihood(X, y, preds, jit)
                if (abs(loglike - LL0) < abs(loglike_old - LL0)) and loglike > LL0:
                    break
                step_size *= 0.5
                coef -= step_size
            if abs(loglike - LL0) <= tol:
                ci.append(coef[coef_idx])
                break
        if abs(loglike - LL0) > tol:
            ci.append(np.nan)
            warning_msg = (
                f"Non-converged PL confidence limits - max number of "
                f"iterations exceeded for variable x{coef_idx}. Try "
                f"increasing pl_max_iter."
            )
            warnings.warn(warning_msg, ConvergenceWarning, stacklevel=2)
    return ci


def load_sex2():
    """
    Load the sex2 dataset from `logistf`.

    Returns
    -------
    X
        sex2 data as numpy array
    y
        sex2 `case` target column
    feature_names
        List of feature names

    References
    ----------
    Cytel Inc., (2010) LogXact 9 user manual, Cambridge, MA:Cytel Inc
    """
    with open_text("firthlogist.datasets", "sex2.csv") as sex2:
        X = np.loadtxt(sex2, skiprows=1, delimiter=",")
    y = X[:, 0]
    X = X[:, 1:]
    feature_names = ["age", "oc", "vic", "vicl", "vis", "dia"]
    return X, y, feature_names


def load_endometrial():
    """
    Load the endometrial cancer dataset analyzed in Heinze and Schemper (2002). The data
    was originally provided by Dr E. Asseryanis from the Vienna University Medical
    School

    Returns
    -------
    X
        endometrial data as numpy array
    y
        endometrial `HG` target column
    feature_names
        List of feature names

    References
    ----------
    Agresti, A (2015). Foundations of Linear and Generalized Linear Models.
    Wiley Series in Probability and Statistics.

    Heinze G, Schemper M (2002). A solution to the problem of separation in logistic
    regression. Statistics in Medicine 21: 2409-2419.
    """
    with open_text("firthlogist.datasets", "endometrial.csv") as sex2:
        X = np.loadtxt(sex2, skiprows=1, delimiter=",")
    y = X[:, -1]
    X = X[:, :-1]
    feature_names = ["NV", "PI", "EH"]
    return X, y, feature_names


In [86]:
X = np.load('../letter_img_X.npy')
y = np.load('../letter_img_y.npy')
fl_jit = FirthLogisticRegression(jit=True)
fl_nojit = FirthLogisticRegression(jit=False)
fl_nojit.fit(X, y)
fl_nojit.summary()

                  coef    std err       [0.025       0.975]       p-value
---------  -----------  ---------  -----------  -----------  ------------
x1          -0.195116   0.0866002   -0.366847    -0.0259218  0.021921
x2          -0.421939   0.0381293   -0.497584    -0.347674   2.29921e-30
x3          -0.426402   0.0961199   -0.616994    -0.238286   7.67488e-06
x4           0.176908   0.0514913    0.0758403    0.278311   0.00055621
x5           0.33899    0.0820833    0.178098     0.501285   3.39404e-05
x6           0.978589   0.0784099    0.825111     1.13497    1.2355e-38
x7          -0.0481063  0.0732158   -0.193543     0.0952611  0.503734
x8          -0.331244   0.0521999   -0.436036    -0.22988    3.09936e-11
x9           1.13453    0.0528053    1.03265      1.24074    1.80347e-161
x10          0.868236   0.0472195    0.777148     0.963157   4.02882e-109
x11          0.0504533  0.0716943   -0.0909186    0.192164   0.479054
x12          0.0844147  0.052282    -0.0189676    0.18745 

'                  coef    std err       [0.025       0.975]       p-value\n---------  -----------  ---------  -----------  -----------  ------------\nx1          -0.195116   0.0866002   -0.366847    -0.0259218  0.021921\nx2          -0.421939   0.0381293   -0.497584    -0.347674   2.29921e-30\nx3          -0.426402   0.0961199   -0.616994    -0.238286   7.67488e-06\nx4           0.176908   0.0514913    0.0758403    0.278311   0.00055621\nx5           0.33899    0.0820833    0.178098     0.501285   3.39404e-05\nx6           0.978589   0.0784099    0.825111     1.13497    1.2355e-38\nx7          -0.0481063  0.0732158   -0.193543     0.0952611  0.503734\nx8          -0.331244   0.0521999   -0.436036    -0.22988    3.09936e-11\nx9           1.13453    0.0528053    1.03265      1.24074    1.80347e-161\nx10          0.868236   0.0472195    0.777148     0.963157   4.02882e-109\nx11          0.0504533  0.0716943   -0.0909186    0.192164   0.479054\nx12          0.0844147  0.052282    -0.01896

In [80]:
%timeit fl_jit.fit(X, y)
%timeit fl_nojit.fit(X, y)

2.02 s ± 108 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.28 s ± 228 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [81]:
%timeit fl_jit.fit(X, y)
%timeit fl_nojit.fit(X, y)

2.11 s ± 147 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.03 s ± 71.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [2]:
def _get_XW(X, preds, mask=None):
    # mask is 1-indexed because 0 == None
    rootW = np.sqrt(preds * (1 - preds))
    XW = rootW[..., np.newaxis] * X

    # is this equivalent??
    # https://github.com/georgheinze/logistf/blob/master/src/logistf.c#L150-L159
    if mask:
        XW[:, mask - 1] = 0
    return XW

def _get_aug_XW(X, preds, tau, h_diag):
    # mask is 1-indexed because 0 == None
#     rootW = np.sqrt(preds * (1 - preds) * (1 + 2 * tau * h_diag))
    rootW = np.sqrt(preds * (1 - preds) * (1 + h_diag))
#     rootW = np.sqrt(preds * (1 - preds))
    XW = rootW[..., np.newaxis] * X
    
    
    return XW

def _get_aug_XW_jon(X, preds, hats):
    rootW = np.sqrt(preds * (1 - preds) * (1 + hats))
    XW = rootW[:, np.newaxis] * X
    return XW


def _hat_diag(XW):
    # Get diagonal elements of the hat matrix
    # Q = np.linalg.qr(XW, mode="reduced")[0]
    qr, tau, _, _ = lapack.dgeqrf(XW)
    Q, _, _ = lapack.dorgqr(qr, tau)
    hat = np.einsum("ij,ij->i", Q, Q)
    return hat

def _loglikelihood(X, y, preds):
    # penalized log-likelihood
    XW = _get_XW(X, preds)
    fisher_info_mtx = XW.T @ XW
    penalty = 0.5 * np.log(np.linalg.det(fisher_info_mtx))
    return -1 * (np.sum(y * np.log(preds) + (1 - y) * np.log(1 - preds)) + penalty)

def _get_preds(X, coef):
    return expit(X @ coef)

In [ ]:
sex2 = pd.read_csv('tests/sex2.csv')
# diabetes = pd.read_csv('tests/diabetes.csv')
# diabetes.insert(0, 'intercept', np.repeat(1, diabetes.shape[0]))
X = sex2.iloc[:, 1:].values
y = sex2['case'].values
# X = diabetes.iloc[:, :-1].values
# y = diabetes['Outcome'].values
firth = FirthLogisticRegression()
firth.fit(X, y)

In [40]:
X = np.hstack((np.ones((X.shape[0], 1)), X))
coef = np.array([firth.intercept_, *firth.coef_])

In [41]:
k = X.shape[1]
n = X.shape[0]
which = -1
iSel = 1
tol = firth.tol
tau = 0.5
alpha = 0.05
max_halfstep = 1000
max_stepsize = 5
max_iter = 25

In [42]:
# Initialization
LL0 = firth.loglik_ - (chi2.ppf(1 - alpha, 1) / 2)
preds = _get_preds(X, coef)
loglik = -_loglikelihood(X, y, preds)
loglik

-132.5393795328482

# First Iteration

In [43]:
# calc XW^1/2
xw2 = _get_XW(X, preds).T
fisher_info_matrix = xw2 @ xw2.T
log_fisher_det = np.log(np.linalg.det(fisher_info_matrix))
fisher_inv = np.linalg.inv(fisher_info_matrix)

In [44]:
# Calc hat diag and loglikelihood
tmpNxK = xw2.T @ fisher_inv
# This looks correct
h_diag = np.diag(tmpNxK @ xw2)
h_diag.shape

(239,)

In [47]:
# Get the augmented data using hat diag (lines 753-759 in C code)
xw2_aug = _get_aug_XW(X, preds, tau, h_diag).T
fisher_aug = xw2_aug @ xw2_aug.T
fisher_aug_det = np.log(np.linalg.det(fisher_aug))
fisher_aug_inv = np.linalg.inv(fisher_aug)

In [48]:
xw2_aug

array([[0.47334627, 0.27739598, 0.27739598, ..., 0.42014777, 0.36321753,
        0.36321753],
       [0.        , 0.        , 0.        , ..., 0.42014777, 0.36321753,
        0.36321753],
       [0.        , 0.        , 0.        , ..., 0.42014777, 0.36321753,
        0.36321753],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.36321753,
        0.36321753],
       [0.47334627, 0.        , 0.        , ..., 0.        , 0.36321753,
        0.36321753],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

True

In [24]:
# calc the new weights
w = (y - preds) + 2 * tau * h_diag * (0.5 - preds)
w[:5]

array([0.67250885, 0.07458449, 0.07458449, 0.07458449, 0.07458449])

In [25]:
# Calc Ustar and lambda values
Ustar = X.T @ w
print(Ustar)
fisher_inv = -fisher_aug_inv.copy()
tmpKx1 = Ustar.T @ fisher_inv
tmp1x1 = tmpKx1.T @ Ustar
tmp1x1

[ 1.99780044e-06  1.94686835e-06 -1.47840247e-07  2.15588820e-06
  2.28431038e-06  2.07192717e-06  1.81926204e-06]


-9.43491787769868e-12

In [26]:
fisher_aug_inv[iSel, iSel]

0.179487905010124

In [27]:
under_root = -2 * ((LL0 - loglik) + 0.5 * tmp1x1) / fisher_aug_inv[iSel, iSel]
print(under_root)
lambda_ = 0 if under_root < 0 else which * np.sqrt(under_root)
print(lambda_)
Ustar[iSel] = Ustar[iSel] + lambda_
step_size = fisher_aug_inv.T @ Ustar
if max_halfstep >= 0:
    mx = np.max(np.abs(step_size)) / max_halfstep
    if mx > 1:
        step_size = step_size / mx
new_coef = coef + step_size

21.402326917164384
-4.626264899156163


In [28]:
step_size

array([ 0.12688378, -0.83035839, -0.00841051,  0.10717171, -0.16004898,
       -0.0033202 ,  0.30335563])

In [30]:
loglik_old = loglik.copy()
for halfs in range(1, max_halfstep + 1):
    new_preds = _get_preds(X, new_coef)
    xw2 = _get_XW(X, new_preds).T
    loglik = -_loglikelihood(X, y, new_preds)
    fisher_info_matrix = xw2 @ xw2.T
    fisher_det = np.linalg.det(fisher_info_matrix)
    fisher_inv = np.linalg.inv(fisher_info_matrix)
    # Calc hat diag and loglikelihood
    tmpNxK = xw2.T @ fisher_inv
    h_diag = np.diagonal(tmpNxK @ xw2)
    loglik = -_loglikelihood(X, y, new_preds)
    # Get augmented data
    xw2_aug = _get_aug_XW(X, new_preds, tau, h_diag).T
    fisher_aug = xw2_aug @ xw2_aug.T
    fisher_aug_det = np.linalg.det(fisher_aug)
    fisher_aug = np.linalg.inv(fisher_aug)
    if (abs(loglik - LL0) < abs(loglik_old - LL0)) and (loglik > LL0):
        print(halfs, 'broken out of loop')
        break
    step_size /= 2
    new_coef -= step_size
if abs(loglik - LL0) <= tol:
    print(new_coef[iSel])

In [31]:
step_size

array([ 1.18416015e-302, -7.74943275e-302, -7.84922730e-304,
        1.00019462e-302, -1.49367890e-302, -3.09862069e-304,
        2.83110773e-302])

# Iteration To Max Iters

In [ ]:
sex2 = pd.read_csv('tests/sex2.csv')
diabetes = pd.read_csv('tests/diabetes.csv')
# diabetes.insert(0, 'intercept', np.repeat(1, diabetes.shape[0]))
X = sex2.iloc[:, 1:].values
y = sex2['case'].values
# X = diabetes.iloc[:, :-1].values
# y = diabetes['Outcome'].values
firth = FirthLogisticRegression()
firth.fit(X, y)
X = np.hstack((np.ones((X.shape[0], 1)), X))

In [75]:
import sklearn
iris = sklearn.datasets.load_iris()
X, y = iris.data, iris.target
# X = np.hstack((np.ones((X.shape[0], 1)), X))
y = np.where(y == y.flat[0], y, y.flat[0] + 1)
firth = FirthLogisticRegression(skip_ci=True, max_iter=100)
firth.fit(X, y)
firth.coef_
firth.n_iter_

36

In [92]:
df = pd.DataFrame(X, columns=['x0','x1','x2','x3'])
df['y'] = y
df.to_csv('iris.csv', index=False)

In [93]:
np.finfo(float).eps

2.220446049250313e-16

In [22]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(penalty='none')
lr.fit(X, y)
lr.coef_

array([[-2.02162242, -6.99849918, 11.14813559,  5.15488554]])

In [69]:
# coef = np.array([firth.intercept_, *firth.coef_])
# X = np.hstack((np.ones((X.shape[0], 1)), X))
coef = np.array([*firth.coef_, firth.intercept_])
X = np.hstack((X, np.ones((X.shape[0], 1))))

In [70]:
k = X.shape[1]
n = X.shape[0]
# which = -1 -> lower bounds, which = 1 -> upper bounds
which = -1
tol = firth.tol
tau = 0.5
alpha = 0.05
max_halfstep = 1000
max_stepsize = 5
max_iter = 25

In [28]:
firth.intercept_

8.337338455265538

In [41]:
k

4

In [73]:
lower_bds = []
upper_bds = []
for i in range(1, k):
    iSel = i
    LL0 = firth.loglik_ - (chi2.ppf(1 - alpha, 1) / 2)
    coef = np.array([*firth.coef_, firth.intercept_])

    for iter_ in range(1, max_iter + 1):
        # Initialization

        preds = _get_preds(X, coef)
        loglik = -_loglikelihood(X, y, preds)
#         print(f'loglik = {loglik}')
        # print(f'{iter_=}')
        # calc XW^1/2
        xw2 = _get_XW(X, preds).T
        fisher_info_matrix = xw2 @ xw2.T
        log_fisher_det = np.log(np.linalg.det(fisher_info_matrix))
        fisher_inv = np.linalg.inv(fisher_info_matrix)

        # Calc hat diag and loglikelihood
        tmpNxK = xw2.T @ fisher_inv
        # This looks correct
        h_diag = np.diag(tmpNxK @ xw2)

        # Get the augmented data using hat diag (lines 753-759 in C code)
        xw2_aug = _get_aug_XW(X, preds, tau, h_diag).T
        fisher_aug = xw2_aug @ xw2_aug.T
        fisher_aug_det = np.log(np.linalg.det(fisher_aug))
        fisher_aug_inv = np.linalg.inv(fisher_aug)
#         print(xw2_aug.T)
        # calc the new weights
        w = (y - preds) + 2 * tau * h_diag * (0.5 - preds)
        Ustar = X.T @ w
        # print(f'{Ustar=}')
        fisher_inv = -fisher_aug_inv.copy()
        tmpKx1 = Ustar.T @ fisher_inv
        tmp1x1 = tmpKx1.T @ Ustar
#         print(Ustar)
#         print(fisher_inv)
#         print(Ustar)
#         print(f'{tmp1x1=}')
        # print(f'fisher_augmented = {fisher_aug_inv[iSel, iSel]}')
        under_root = -2 * ((LL0 - loglik) + 0.5 * tmp1x1) / fisher_aug_inv[iSel, iSel]
#         print(f'{under_root=}')
#         print((LL0 - loglik) + 0.5 * tmp1x1)
        lambda_ = 0 if under_root < 0 else which * np.sqrt(under_root)
#         print(f'{lambda_=}')
        Ustar[iSel] = Ustar[iSel] + lambda_
        step_size = fisher_aug_inv.T @ Ustar
        if max_halfstep >= 0:
            mx = np.max(np.abs(step_size)) / max_halfstep
            if mx > 1:
                step_size = step_size / mx
#         print(f'step_size={[x for x in step_size]}\n')
#         print('coef_old', coef)
        coef = coef + step_size
#         print('coef', coef)
        # Iterate over halfsteps
        loglik_old = loglik.copy()
        for halfs in range(1, max_halfstep + 1):
            preds = _get_preds(X, coef)
            xw2 = _get_XW(X, preds).T
            loglik = -_loglikelihood(X, y, preds)
            fisher_info_matrix = xw2 @ xw2.T
            fisher_det = np.linalg.det(fisher_info_matrix)
            fisher_inv = np.linalg.inv(fisher_info_matrix)
            # Calc hat diag and loglikelihood
            tmpNxK = xw2.T @ fisher_inv
            h_diag = np.diagonal(tmpNxK @ xw2)
            loglik = -_loglikelihood(X, y, preds)
            # Get augmented data
            xw2_aug = _get_aug_XW(X, preds, tau, h_diag).T
            fisher_aug = xw2_aug @ xw2_aug.T
            fisher_aug_det = np.linalg.det(fisher_aug)
            fisher_aug = np.linalg.inv(fisher_aug)
#             print('loglik-ll0=', abs(loglik-LL0))
            if (abs(loglik - LL0) < abs(loglik_old - LL0)) and (loglik > LL0):
                break
            step_size /= 2
            coef -= step_size
#             print('posths_coef', coef)
        if abs(loglik - LL0) <= tol:
            lower_bds.append(coef[iSel])
            break
#         print()

In [74]:
lower_bds

[-14.915308156930347]

In [38]:
lower_bds

[-0.8185450801110972,
 -1.9737883819057844,
 -0.941424219446236,
 1.2730351899978471,
 -3.260839223819344,
 -1.608086648992101,
 0.7746070226955406]

In [38]:
rootW_h = np.sqrt(preds * (1 - preds) * (1 + 2 * tau * 0))
rootW = np.sqrt(preds * (1 - preds))
np.allclose(rootW, rootW_h)

True

In [37]:
(1 + 2 * tau * -1)

0.0